In [29]:
import torch
import os
import torchvision
import numpy
import matplotlib.pyplot as plt
from torchvision import datasets, transforms, models
import resnet50
from sklearn.metrics import f1_score
Path_model = 'model_resnet/first_model_resnet.pt'

In [30]:
#import model resnet
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet50.Resnet_model()
model = model.model


In [31]:
model = torch.load(Path_model,map_location=torch.device('cpu'))

In [32]:
#model.load_state_dict(torch.load(Path_model, map_location=torch.device('cpu')))

In [33]:
# using test dataset and leaving val for the end(should have been the opposite)
transforms_train = transforms.Compose([transforms.Resize(225),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.5, 0.5, 0.5],
                                                            [0.5, 0.5, 0.5])])

transforms_test = transforms.Compose([transforms.Resize(225),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.5, 0.5, 0.5],
                                                            [0.5, 0.5, 0.5])])

train_data = datasets.ImageFolder(root='data\FIRE-SMOKE-DATASET\Train', transform=transforms_train)
test_data = datasets.ImageFolder(root='data\FIRE-SMOKE-DATASET\Test', transform=transforms_test)
print(len(train_data), len(test_data))

#spliting the data
train_data, val_data = torch.utils.data.random_split(train_data, [2400,300])

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=True)
test_data_loader = torch.utils.data.DataLoader(test_data,  batch_size=64, shuffle=True)

classes =('Fire', 'Neutral', 'Smoke')


2700 300


In [35]:

# calculating f1 score for the 5 models
#Accuracy  and F1 score
from sklearn.metrics import f1_score
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_data_loader:
        output = model(images)
        proba, predicted = torch.max(output, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        f1_score_conv = f1_score(predicted.detach().numpy(), labels.detach().numpy(),average='micro')
        '''
        average:
        This parameter is required for multiclass/multilabel targets.
        If None, the scores for each class are returned. 
        Otherwise, this determines the type of averaging performed on the data
        '''
print (f'accuracy test set: {100 * correct/total}')
print (f'F1 score test set: {f1_score_conv}')




accuracy test set: 92.66666666666667
F1 score test set: 1.0


In [ ]:
# accuracy per each class
correct = { a: 0 for a in classes}
total =  { a: 0 for a in classes}

with torch.no_grad():
    for images, labels in test_data_loader:
        output = model(images)
        proba, predicted = torch.max(output, dim=1)
        for label, prediction in zip(labels,predicted):
            if label == prediction :
                correct[classes[label]] +=1
            total [classes[label]]+=1
                
        
for c, k in correct.items():
   print (f'accuracy test set class {c}: {100 * k/total[c]}')

In [ ]:
# display an image from the test set
dataiter = iter(test_data_loader)
images, labels = dataiter.next()
images = images[:4]
labels = labels[:4]
#print images

def imshow(img):
    img = img / 2 + 0.5 #unnormalize
    npimg = img.numpy()
    plt.imshow(numpy.transpose(npimg,(1,2,0)))
    plt.show()
imshow(torchvision.utils.make_grid(images))

In [ ]:
#%%
print('Truth:', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
dataiter = iter(train_data_loader)
images, labels = dataiter.next()
print(images.shape)
classes[labels[0]]

outputs = net(images)
a , predicted = torch.max(outputs, 1)
print('Predicted:', ' '.join('%5s' % classes[labels[j]] for j in range(4)))
# %%
labels.size()
# %%

In [ ]:
def load_checkpoint(model, optimizer,filename='model_resnet/first_model_resnet.pt'):
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    start_epoch = 0
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
                  .format(filename, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    return model, optimizer, start_epoch

model, optimizer, start_epoch = load_checkpoint(model)

                        
import copy

#model.load_state_dict(copy.deepcopy(torch.load('model_resnet/first_model_resnet.pt',device)))